In [ ]:
import numpy as np

import os
import gzip
import matplotlib.pyplot as plt
import sys
import time


import argparse
import sys
#import tempfile

FLAGS = None

In [ ]:
import tensorflow as tf

In [ ]:
import h5py
#import numpy as np
import random

class Data:
  def __init__(self):
    with h5py.File("cell_data.h5", "r") as data:
      self.train_images = [data["/train_image_{}".format(i)][:] for i in range(28)]
      self.train_labels = [data["/train_label_{}".format(i)][:] for i in range(28)]
      self.test_images = [data["/test_image_{}".format(i)][:] for i in range(3)]
      self.test_labels = [data["/test_label_{}".format(i)][:] for i in range(3)]
    
    self.input_resolution = 300
    self.label_resolution = 116

    self.offset = (300 - 116) // 2

  def get_train_image_list_and_label_list(self):
    n = random.randint(0, len(self.train_images) - 1)
    x = random.randint(0, (self.train_images[n].shape)[1] - self.input_resolution - 1)
    y = random.randint(0, (self.train_images[n].shape)[0] - self.input_resolution - 1)
    image = self.train_images[n][y:y + self.input_resolution, x:x + self.input_resolution, :]

    x += self.offset
    y += self.offset
    label = self.train_labels[n][y:y + self.label_resolution, x:x + self.label_resolution]
    
    return [image], [label]

  def get_test_image_list_and_label_list(self):
    coord_list = [[0,0], [0, 116], [0, 232], 
                  [116,0], [116, 116], [116, 232],
                  [219,0], [219, 116], [219, 232]]
    
    image_list = []
    label_list = []
    
    for image_id in range(3):
      for y, x in coord_list:
        image = self.test_images[image_id][y:y + self.input_resolution, x:x + self.input_resolution, :]
        image_list.append(image)
        x += self.offset
        y += self.offset
        label = self.test_labels[image_id][y:y + self.label_resolution, x:x + self.label_resolution]
        label_list.append(label)
    

    return image_list, label_list

In [ ]:
d = Data()
print(len(d.train_images))
print(d.train_images[0].shape)
im, lab = d.get_train_image_list_and_label_list()
print(im[0].shape)
print(len(lab))
print(lab[0].shape)
print(lab[0])
plt.imshow(lab[0], cmap='gray')
plt.show()
tmp=np.reshape(im[0],[300,300])
plt.imshow(tmp, cmap='gray')
plt.show()



im, lab = d.get_test_image_list_and_label_list()

print(len(lab))
print(lab[0].shape)
print(lab[0])
plt.imshow(lab[0], cmap='gray')
plt.show()
tmp=np.reshape(im[0],[300,300])
plt.imshow(tmp, cmap='gray')
plt.show()

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
    
    #v = tf.get_variable("v", shape=shape, initializer=tf.glorot uniform initializer())
    #return v
    

def bias_variable(shape):
    initial = tf.constant(0.0, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='VALID')

def print_size(l):
    print(str(l)+' size=',l.get_shape().as_list())

In [ ]:
def u_net(x):
    mini_batch_size = 1
    # Reshape the image
    print_size(x)
    
    x_image = tf.reshape(x, [mini_batch_size, 300, 300, 1])
    print_size(x_image)
    
    # convolution 1. Output 298*298*32
    W_conv1 = weight_variable([3, 3, 1, 32])
    b_conv1 = bias_variable([32])    
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    print_size(h_conv1)
    
    # convolution 2. Output 296*296*32
    W_conv2 = weight_variable([3, 3, 32, 32])
    b_conv2 = bias_variable([32])    
    h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)
    print_size(h_conv2)
    
    # max pool 3. Output 148*148*32
    h_pool3 = max_pool_2x2(h_conv2)
    print_size(h_pool3)
    
    # convolution 4. Output 146*146*64
    W_conv4 = weight_variable([3, 3, 32, 64])
    b_conv4 = bias_variable([64])     
    h_conv4 = tf.nn.relu(conv2d(h_pool3, W_conv4) + b_conv4)
    print_size(h_conv4)
    
    # convolution 5. Output 144*144*64
    W_conv5 = weight_variable([3, 3, 64, 64])
    b_conv5 = bias_variable([64])     
    h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5) + b_conv5)
    print_size(h_conv5)
    
    # max pool 6. Output 72*72*64
    h_pool6 = max_pool_2x2(h_conv5)
    print_size(h_pool6)
    
    # convolution 7. Output 70*70*128
    W_conv7 = weight_variable([3, 3, 64, 128])
    b_conv7 = bias_variable([128])     
    h_conv7 = tf.nn.relu(conv2d(h_pool6, W_conv7) + b_conv7)
    print_size(h_conv7)
    
    # convolution 8. Output 68*68*128
    W_conv8 = weight_variable([3, 3, 128, 128])
    b_conv8 = bias_variable([128])     
    h_conv8 = tf.nn.relu(conv2d(h_conv7, W_conv8) + b_conv8)
    print_size(h_conv8)
    
    # max pool 9. Output 34*34*128
    h_pool9 = max_pool_2x2(h_conv8)
    print_size(h_pool9)
    
    # convolution 10. Output 32*32*256
    W_conv10 = weight_variable([3, 3, 128, 256])
    b_conv10 = bias_variable([256])     
    h_conv10 = tf.nn.relu(conv2d(h_pool9, W_conv10) + b_conv10)
    print_size(h_conv10)
    
    # convolution 11. Output 30*30*256
    W_conv11 = weight_variable([3, 3, 256, 256])
    b_conv11 = bias_variable([256])     
    h_conv11 = tf.nn.relu(conv2d(h_conv10, W_conv11) + b_conv11)
    print_size(h_conv11)
    
    # max pool 12. Output 15*15*256
    h_pool12 = max_pool_2x2(h_conv11)
    print_size(h_pool12)
    
    # convolution 13. Output 13*13*512
    W_conv13 = weight_variable([3, 3, 256, 512])
    b_conv13 = bias_variable([512])     
    h_conv13 = tf.nn.relu(conv2d(h_pool12, W_conv13) + b_conv13)
    print_size(h_conv13)
    
    # convolution 14. Output 11*11*512
    W_conv14 = weight_variable([3, 3, 512, 512])
    b_conv14 = bias_variable([512])     
    h_conv14 = tf.nn.relu(conv2d(h_conv13, W_conv14) + b_conv14)
    print_size(h_conv14)
    
    # up-convolition 15. Output 22*22*256
    W_uconv15 = weight_variable([2, 2, 256, 512])   
    stride  = [1, 2, 2, 1]
    h_uconv15 = tf.nn.conv2d_transpose(h_conv14, W_uconv15,[mini_batch_size, 22, 22, 256],stride, padding='VALID')
    print_size(h_uconv15)
    # concat with croped copy from layer 11. Output 22*22*512
    h_conv11_crop = tf.image.resize_image_with_crop_or_pad(h_conv11, 22, 22)
    print_size(h_conv11_crop)
    #print(h_conv11_crop.get_shape().as_list())
    h_uconv15_ = tf.concat([h_conv11_crop, h_uconv15], 3)
    print_size(h_uconv15_)
    
    # convolution 16. Output 20*20*256
    W_conv16 = weight_variable([3, 3, 512, 256])
    b_conv16 = bias_variable([256])     
    h_conv16 = tf.nn.relu(conv2d(h_uconv15_, W_conv16) + b_conv16)
    print_size(h_conv16)
    
    # convolution 17. Output 18*18*256
    W_conv17 = weight_variable([3, 3, 256, 256])
    b_conv17 = bias_variable([256])     
    h_conv17 = tf.nn.relu(conv2d(h_conv16, W_conv17) + b_conv17)
    print_size(h_conv17)
    
    # up-convolition 18. Output 36*36*128
    W_uconv18 = weight_variable([2, 2, 128, 256])   
    stride  = [1, 2, 2, 1]
    h_uconv18 = tf.nn.conv2d_transpose(h_conv17, W_uconv18,[mini_batch_size, 36, 36, 128],stride, padding='VALID') 
    print_size(h_uconv18)
    # concat with croped copy from layer 8. Output 36*36*256
    h_conv8_crop = tf.image.resize_image_with_crop_or_pad(h_conv8, 36, 36)
    print_size(h_conv8_crop)
    h_uconv18_ = tf.concat([h_conv8_crop, h_uconv18], 3)
    print_size(h_uconv18_)
    
    # convolution 19. Output 34*34*128
    W_conv19 = weight_variable([3, 3, 256, 128])
    b_conv19 = bias_variable([128])     
    h_conv19 = tf.nn.relu(conv2d(h_uconv18_, W_conv19) + b_conv19)
    print_size(h_conv19)
    
    # convolution 20. Output 32*32*128
    W_conv20 = weight_variable([3, 3, 128, 128])
    b_conv20 = bias_variable([128])     
    h_conv20 = tf.nn.relu(conv2d(h_conv19, W_conv20) + b_conv20)
    print_size(h_conv20)
    
    
    # up-convolition 21. Output 64*64*64
    W_uconv21 = weight_variable([2, 2, 64, 128])   
    stride  = [1, 2, 2, 1]
    h_uconv21 = tf.nn.conv2d_transpose(h_conv20, W_uconv21,[mini_batch_size, 64, 64, 64],stride, padding='VALID') 
    print_size(h_uconv21)
    # concat with croped copy from layer 5. Output 64*64*128
    h_conv5_crop = tf.image.resize_image_with_crop_or_pad(h_conv5, 64, 64)
    print_size(h_conv5_crop)
    h_uconv21_ = tf.concat([h_conv5_crop, h_uconv21], 3)
    print_size(h_uconv21_)
    
    # convolution 22. Output 62*62*64
    W_conv22 = weight_variable([3, 3, 128, 64])
    b_conv22 = bias_variable([64])     
    h_conv22 = tf.nn.relu(conv2d(h_uconv21_, W_conv22) + b_conv22)
    print_size(h_conv22)
    
    # convolution 23. Output 60*60*64
    W_conv23 = weight_variable([3, 3, 64, 64])
    b_conv23 = bias_variable([64])     
    h_conv23 = tf.nn.relu(conv2d(h_conv22, W_conv23) + b_conv23)
    print_size(h_conv23)
    
    # up-convolition 24. Output 120*120*32
    W_uconv24 = weight_variable([2, 2, 32, 64])   
    stride  = [1, 2, 2, 1]
    h_uconv24 = tf.nn.conv2d_transpose(h_conv23, W_uconv24,[mini_batch_size, 120, 120, 32],stride, padding='VALID')
    print_size(h_uconv24)
    # concat with croped copy from layer 2. Output 120*120*64
    h_conv2_crop = tf.image.resize_image_with_crop_or_pad(h_conv2, 120, 120)
    print_size(h_conv2_crop)
    h_uconv24_ = tf.concat([h_conv2_crop, h_uconv24], 3)
    print_size(h_uconv24_)
    
    # convolution 25. Output 118*118*32
    W_conv25 = weight_variable([3, 3, 64, 32])
    b_conv25 = bias_variable([32])     
    h_conv25 = tf.nn.relu(conv2d(h_uconv24_, W_conv25) + b_conv25)
    print_size(h_conv25)
    
    # convolution 26. Output 116*116*32
    W_conv26 = weight_variable([3, 3, 32, 32])
    b_conv26 = bias_variable([32])     
    h_conv26 = tf.nn.relu(conv2d(h_conv25, W_conv26) + b_conv26)
    print_size(h_conv26)
    
    # convolution 27. Output 116*116*2
    W_conv27 = weight_variable([1, 1, 32, 2])
    b_conv27 = bias_variable([2])     
    h_conv27 = conv2d(h_conv26, W_conv27) + b_conv27
    print_size(h_conv27)
    
    
    return h_conv27


In [ ]:
acc_tr_array=[]
acc_te_array=[]    
loss_tr_array=[]
loss_te_array=[]

mini_batch_size=1
learning_rate=0.0001


x = tf.placeholder(tf.float32, [None, 300,300,1])
y = tf.placeholder(tf.int32, [None, 116,116])




y_hat = u_net(x)

y_resh = tf.reshape(y, [mini_batch_size*116*116])
y_hot=tf.one_hot(y_resh,2)

print('y_hot', y_hot.get_shape().as_list())


y_hat_resh = tf.reshape(y_hat, [mini_batch_size*116*116, 2])

print('y_hat_resh',y_hat_resh.get_shape().as_list())




loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_hot, logits=y_hat_resh)

loss = tf.reduce_mean(loss)



train_step = tf.train.AdamOptimizer(learning_rate, 0.95, 0.99).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_hat_resh, 1), tf.argmax(y_hot, 1))
correct_prediction = tf.cast(correct_prediction, tf.float32)



config = tf.ConfigProto(
device_count = {'GPU': 0})
sess = tf.Session(config=config)


with sess as sess:
    sess.run(tf.global_variables_initializer())
    t0 = time.time()
    for i in range(40001):
        batch = d.get_train_image_list_and_label_list()
        
        train_step.run(feed_dict={x: batch[0], y: batch[1]})
       
   
        
        if i % 500 == 0:
            prediction = correct_prediction.eval(feed_dict={x: batch[0], y: batch[1]})
            #print('prediction.shape',prediction.shape)
            train_accuracy = (np.sum(prediction))/(2*prediction.shape[0]-np.sum(prediction))

            loss_tr = loss.eval(feed_dict = {x: batch[0], y: batch[1]})

            loss_tr_array.append(loss_tr)
            acc_tr_array.append(train_accuracy)
            
            print("Batch: " + str(i) + " Train Loss value:" + str(loss_tr)+" Train Accuracy "+str(train_accuracy))
            
            
            #test
            im, lbl = d.get_test_image_list_and_label_list()
            test_accuracy=0.0
            for j in range(len(lbl)):
                output= y_hat.eval(feed_dict={x: np.reshape(im[j],[1,300,300,1]), y: np.reshape(lbl[j],[1,116,116])})
                #print(output.shape)
                output=np.argmax(output, axis=-1)
                #print(output.shape)
                
                #prediction
                print('Prediction ', j)
                pixels_in = np.reshape(output,[116, 116])
                plt.imshow(pixels_in, cmap='gray')
                plt.show()
                #label
                print('Label ', j)
                pixels_in = np.reshape(lbl[j],[116, 116])
                plt.imshow(pixels_in, cmap='gray')
                plt.show()
                
                prediction_test = correct_prediction.eval(feed_dict={x: np.reshape(im[j],[1,300,300,1]), y: np.reshape(lbl[j],[1,116,116])})
                test_accuracy = test_accuracy+(np.sum(prediction_test))/(2*prediction_test.shape[0]-np.sum(prediction_test))
            
            test_accuracy = test_accuracy/len(lbl)
            acc_te_array.append(test_accuracy)
            print('test_accuracy',test_accuracy)
            

    t1 = time.time()    
    print('Duration: {:.1f}s'.format(t1-t0))
    
    


In [ ]:

print('acc_tr_array', acc_tr_array)  
print('acc_te_array', acc_te_array) 
f_tr = open("train.txt","w") 
f_te = open("test.txt","w")
for t in acc_tr_array:
    f_tr.write(str(t)+'\n') 
for t in acc_te_array:
    f_te.write(str(t)+'\n') 
#f_te.write(acc_te_array) 


f_tr.close()
f_te.close()

In [ ]:
#print(valid_acc_arr)


plt.figure(figsize=(12, 8))
x=[]
for i in range(len(acc_te_array)):
    x.append(i*500)
print(len(x))
#print(acc_tr_array)
print(len(acc_te_array))      
plt.plot(x, -np.log(acc_tr_array),label='train acc', linewidth=2.0)
plt.plot(x, -np.log(acc_te_array), label='test acc', linewidth=2.0)
#plt.plot(np.log(loss_tr_array1), label='Log of train loss. Learning rate 1e-1', linewidth=2.0)
plt.legend(loc='best') 
plt.ylabel('Negative log of accuracy')
plt.xlabel('Iterations')
plt.show()    